In [1]:
from inference_utils import example_mega_fast_csv_parsing, parse_addresses_from_csv


model_path = "models/address_parser2/checkpoint-750"
csv_path = "data/ocod_history/OCOD_FULL_2015_10.zip"

import time
start_time = time.time()

results = parse_addresses_from_csv(
    model_path=model_path,
    csv_path=csv_path,
    target_column="Property_Address",
    batch_size=2048
)

end_time = time.time()

print(f"Mega-fast CSV Parsing completed in {end_time - start_time:.2f} seconds")

Loading model from models/address_parser2/checkpoint-750
Model loaded on cuda. Labels: ['B-building_name', 'B-city', 'B-filter_type', 'B-postcode', 'B-street_name', 'B-street_number', 'B-unit_id', 'B-unit_type', 'I-building_name', 'I-city', 'I-filter_type', 'I-postcode', 'I-street_name', 'I-street_number', 'I-unit_id', 'I-unit_type', 'O']


/teamspace/studios/this_studio/enhance_ocod/inference_utils.py:197: DtypeWarning: Columns (16,23,27,29,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Loaded CSV from zip: data/ocod_history/OCOD_FULL_2015_10.zip
Processing 99350 addresses in batches of 2048
Processing batch 1/49
Processing batch 2/49
Processing batch 3/49
Processing batch 4/49
Processing batch 5/49
Processing batch 6/49
Processing batch 7/49
Processing batch 8/49
Processing batch 9/49
Processing batch 10/49
Processing batch 11/49
Processing batch 12/49
Processing batch 13/49
Processing batch 14/49
Processing batch 15/49
Processing batch 16/49
Processing batch 17/49
Processing batch 18/49
Processing batch 19/49
Processing batch 20/49
Processing batch 21/49
Processing batch 22/49
Processing batch 23/49
Processing batch 24/49
Processing batch 25/49
Processing batch 26/49
Processing batch 27/49
Processing batch 28/49
Processing batch 29/49
Processing batch 30/49
Processing batch 31/49
Processing batch 32/49
Processing batch 33/49
Processing batch 34/49
Processing batch 35/49
Processing batch 36/49
Processing batch 37/49
Processing batch 38/49
Processing batch 39/49
Proce

In [6]:
results['results'][0]['entities']

[{'type': 'street_number', 'text': '2', 'start': 0, 'end': 1},
 {'type': 'postcode', 'text': 'W1S 2AT', 'start': 23, 'end': 30}]

In [1]:
import pandas as pd
import zipfile

# Path to the zip file
zip_path = "data/ocod_history/OCOD_FULL_2015_10.zip"

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract the CSV file name inside the zip
    csv_filename = "OCOD_FULL_2015_10.csv"
    
    # Read the CSV file directly from the zip archive
    df = pd.read_csv(zip_ref.open(csv_filename))

# Now df contains the data from the CSV file

/tmp/ipykernel_237026/2465846817.py:13: DtypeWarning: Columns (16,23,27,29,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_ref.open(csv_filename))


In [2]:
df

,Title_Number,Tenure,Property_Address,Price_Paid,District,County,Region,Postcode,Proprietor_Name_1,Company_Registration_No_1,Proprietorship_1,Country_Incorporated_1,Proprietor_1_Address_1,Proprietor_1_Address_2,Proprietor_1_Address_3,Proprietor_Name_2,Company_Registration_No_2,Proprietorship_2,Country_Incorporated_2,Proprietor_2_Address_1,Proprietor_2_Address_2,Proprietor_2_Address_3,Proprietor_Name_3,Company_Registration_No_3,Proprietorship_3,Country_Incorporated_3,Proprietor_3_Address_1,Proprietor_3_Address_2,Proprietor_3_Address_3,Proprietor_Name_4,Company_Registration_No_4,Proprietorship_4,Country_Incorporated_4,Proprietor_4_Address_1,Proprietor_4_Address_2,Proprietor_4_Address_3,Date_Proprietor_Added,Additional_Proprietor_Indicator,Multiple_Address_Indicator
0,100007,Freehold,"2 Mill Street, London (W1S 2AT)",NaN,CITY OF WESTMINSTER,GREATER LONDON,GREATER LONDON,W1S 2AT,SPV ST GEORGE LIMITED,NaN,Limited Company or Public Limited Company,ISLE OF MAN,"2a Lord Street, Douglas, Isle of Man, IM1 2BD",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18-01-2007,N,N
1,100073,Freehold,"11 Stanley Crescent, London (W11 2NA)",NaN,KENSINGTON AND CHELSEA,GREATER LONDON,GREATER LONDON,W11 2NA,US TRUST COMPANY OF DELAWARE,NaN,Corporate Body,U.S.A.,"1100 North King Street, Wilmington, Delaware 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09-07-2015,Y,N
2,100232,Freehold,"56 Bell Street, London (NW1 5BU)",NaN,CITY OF WESTMINSTER,GREATER LONDON,GREATER LONDON,NW1 5BU,ENTRIX CAPITAL LIMITED,NaN,Limited Company or Public Limited Company,BRITISH VIRGIN ISLANDS,"Vanterpool Plaza, 2nd Floor, Wickhams Cay I, R...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03-02-2014,N,N
3,100357,Freehold,"14 Holland Villas Road, London (W14 8BT)",NaN,KENSINGTON AND CHELSEA,GREATER LONDON,GREATER LONDON,W14 8BT,VIVAT TRUSTEES LIMITED,NaN,Limited Company or Public Limited Company,JERSEY,"6 Agar Street, London WC2N 4HN",NaN,NaN,UNION TRUSTEES LIMITED,NaN,Limited Company or Public Limited Company,JERSEY,"6 Agar Street, London WC2N 4HN",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26-03-2014,N,N
4,100396,Freehold,"59 and 60 Pall Mall and 1 Crown Passage, St Ja...",NaN,CITY OF WESTMINSTER,GREATER LONDON,GREATER LONDON,SW1Y 5JH,"THE GOVERNMENT OF QUEBEC, CANADA",NaN,Corporate Body,CANADA,"Quebec Government Office London, 59 Pall Mall,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15-09-2008,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99345,YY9295,Leasehold,"Airspace above 31 Brooksfield, South Kirkby, P...",NaN,WAKEFIELD,WEST YORKSHIRE,YORKS AND HUMBER,WF9 3DL,SOLARFAIR INVESTMENTS LIMITED,NaN,Limited Company or Public Limited Company,JERSEY,"PO Box 781, 8 Church Street, St Helier, Jersey...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12-12-2012,N,N
99346,YY9541,Freehold,"land on the north side of Devilliers Way, Norm...",4500000.0,WAKEFIELD,WEST YORKSHIRE,YORKS AND HUMBER,NaN,ICENI INVESTMENTS LIMITED,NaN,Limited Company or Public Limited Company,GUERNSEY,"Richmond House, St Julians Avenue, St Peter Po...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20-03-2014,N,N
99347,YY9805,Leasehold,"Apartment 2, Merchants Court, East Parade, Bra...",NaN,BRADFORD,WEST YORKSHIRE,YORKS AND HUMBER,BD1 5HE,LIMETREE INVESTMENTS LIMITED,NaN,Limited Company or Public Limited Company,ANGUILLA,"Bundesstrasse 5, P.O. Box 4513, CH-6304 ZUG, S...","Global House, 303 Ballards Lane, London N12 8NP",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15-10-2012,N,N
99348,YY9844,Leasehold,"Apartment 5, Merchants Court, East Parade, Bra..